### install dependencies
only do this once

In [1]:
#! python3 -m venv venv
#! source venv/bin/activate
#! pip3 install -r requirements.txt
#! pip install ipykernel
#! python -m ipykernel install --user
#! pip install -e ./
#! pip freeze

### imports

In [14]:
from builtins import range
import numpy as np
import pandas as pd
import os
import logging

In [15]:
from src.embedding_utilities import euclidean_dist, cosine_dist
from src.embedding import Embedding

In [16]:
logging.basicConfig(encoding='utf-8', level=logging.INFO)

# define paths

In [17]:
#s3_client = boto3.client('s3')
BUCKET_NAME = 'ma-2021-07-word-embeddings'
FILE_NAME_EN = 'glove.6B.300d.txt'
FILE_NAME_DE = 'GloVe_vectors_de_10_test_file.txt '#'GloVe_vectors_de_50000.txt'

In [19]:
glove_path_en = ['../data/glove.6B.100d.txt']
#glove_path_de = ['../data/GloVe_vectors_de_part_01.txt', '../data/GloVe_vectors_de_part_02.txt']
glove_path_de = ['../data/GloVe_vectors_de_50000.txt']#['../data/GloVe_vectors_de.txt']

### the cosine distance does not change when doubeling
cosine distance only takes into account angle (the smaller the higher the similarity.)

euclidian looks at absolute length

In [20]:
a = pd.Series([1,1])
b = pd.Series([1,2])
print(f'euclidian distance {euclidean_dist(a,b)}')
print(f'cosine distance {cosine_dist(a,b)}')

euclidian distance 1.0
cosine distance 0.05131670194948623


In [21]:
a_double = a*2
b_double = b*2
print(f'euclidian distance {euclidean_dist(a_double,b_double)}')
print(f'cosine distance {cosine_dist(a_double,b_double)}')

euclidian distance 2.0
cosine distance 0.05131670194948623


In [22]:
dist, metric = cosine_dist, 'cosine'
#alternative: dist, metric = dist1, 'euclidean'

### load pretrained word vectors
source: https://www.kaggle.com/anindya2906/glove6b

In [23]:
import s3fs
import csv

In [11]:
path_list = [f's3://{BUCKET_NAME}/{FILE_NAME_DE}']
logging.info(f'loading word embeddings word to vec from path {path_list}')
df_embeddings = pd.concat([pd.read_csv(path, sep=' ', index_col=0, header=None, engine='python', quoting=csv.QUOTE_NONE) for path in path_list])
df_embeddings = df_embeddings[df_embeddings.index.notnull()] # es gibt im 50k datensatz eine NaN Zeile, die gedroppt werden sollte
logging.info(f'read in embeddings of shape {df_embeddings.shape}')
df_embeddings.head()

[20/Aug/2021 10:49:47] INFO - loading word embeddings word to vec from path ['s3://ma-2021-07-word-embeddings/GloVe_vectors_de_10_test_file.txt']
[20/Aug/2021 10:49:47] INFO - Found credentials in shared credentials file: ~/.aws/credentials
[20/Aug/2021 10:49:48] INFO - read in embeddings of shape (8, 300)


,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
starke,-0.285092,-0.350144,0.066122,-0.059022,-0.074293,0.105105,-0.091429,-0.379363,-0.471944,-0.038044,...,-0.046795,0.486627,0.092285,0.394796,-0.386349,0.016992,0.315967,0.434802,-0.198200,-0.065318
mittagszeit,0.402801,-0.000529,-0.063135,-0.429702,0.059168,-0.181313,0.278056,-0.885087,-0.528817,0.113312,...,-0.310075,-0.564392,-0.512391,-0.061537,-0.261750,0.100853,0.032682,0.089461,0.310686,0.315627
östlichen,-0.404085,0.184555,-0.099343,0.224999,-0.547287,-0.232835,0.416266,0.546959,-0.405216,0.145357,...,-0.121708,-0.263284,0.427416,-0.200418,0.057937,0.020421,0.055736,0.313623,0.598803,0.050427
pumpen,0.366514,-0.275178,-0.337030,-0.311622,0.410426,0.021424,-0.228994,-0.634361,0.073496,-0.183064,...,0.279162,-0.570037,0.434501,0.387588,-0.044226,0.008883,-0.000685,0.623034,0.041198,0.156304
videomaterial,0.074707,-0.030806,-0.425025,-0.320975,0.097818,-0.086039,0.007920,-0.159249,0.564676,-0.091144,...,-0.086772,-0.206852,0.247326,0.037089,-0.422071,0.102761,-0.270570,-0.842040,-0.023887,0.114175


In [10]:
%time glove_embedding_en = Embedding(language='en', path_list=glove_path_en)

INFO:root:loading word embeddings word to vec from path ['../data/glove.6B.100d.txt']


KeyboardInterrupt: 

In [11]:
# working but incredible slow:
# lokal: ca 10 sec for 400k
# read from aws s3: ca 40 sek for 4k => 4000 sek für 400k => faktor 400 schlechter
#%time glove_embedding_en = Embedding(language='en', path=f's3://{BUCKET_NAME}/{FILE_NAME_EN}')

In [12]:
print(glove_embedding_en.embedding.shape)
glove_embedding_en.embedding.head(1)

(399997, 100)


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
the,-0.038194,-0.24487,0.72812,-0.39961,0.083172,0.043953,-0.39141,0.3344,-0.57545,0.087459,...,0.016215,-0.017099,-0.38984,0.87424,-0.72569,-0.51058,-0.52028,-0.1459,0.8278,0.27062


In [25]:
#1.3 Mio entries => takes about 1000sec on local machine 1309281, 360 secs after
#50k entries => takes 9 sec on local machine before, 6 secs after

#50k entries => takes about 9 mins when reading linewise from aws

# aws
#%time glove_embedding_de = Embedding(language='de', path_list = [f's3://{BUCKET_NAME}/{FILE_NAME_DE}'])

#lokal
%time glove_embedding_de = Embedding(language='de', path_list=glove_path_de)

[20/Aug/2021 10:38:13] INFO - loading word embeddings word to vec from path ['../data/GloVe_vectors_de_10_test_file.txt']
[20/Aug/2021 10:38:13] INFO - read in embeddings of shape (8, 300)
[20/Aug/2021 10:38:13] INFO - reading embedding
[20/Aug/2021 10:38:13] INFO - reading index_to_word
total number of entries found:  8. Dimension: 300
CPU times: user 36.2 ms, sys: 2.54 ms, total: 38.7 ms
Wall time: 37.2 ms


In [26]:
print(glove_embedding_de.embedding.shape)
glove_embedding_de.embedding.head(2)

(8, 300)


,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
starke,-0.285092,-0.350144,0.066122,-0.059022,-0.074293,0.105105,-0.091429,-0.379363,-0.471944,-0.038044,...,-0.046795,0.486627,0.092285,0.394796,-0.386349,0.016992,0.315967,0.434802,-0.198200,-0.065318
mittagszeit,0.402801,-0.000529,-0.063135,-0.429702,0.059168,-0.181313,0.278056,-0.885087,-0.528817,0.113312,...,-0.310075,-0.564392,-0.512391,-0.061537,-0.261750,0.100853,0.032682,0.089461,0.310686,0.315627


In [15]:
print(glove_embedding_en.word2vec['tree'][:10])
print(glove_embedding_de.word2vec['baum'][:10])

TypeError: unhashable type: 'slice'

In [ ]:
print(glove_embedding_en.index_to_word[1])
print(glove_embedding_de.index_to_word[1])

In [ ]:
print(glove_embedding_en.embedding.shape)
print(glove_embedding_de.embedding.shape)

# Best of

In [16]:
glove_embedding_en.find_analogies('man', 'fun', 'woman')

man - fun = woman - kids


'kids'

In [17]:
glove_embedding_de.find_analogies('mann', 'spass', 'frau')

mann - spass = frau - schwiegermutter


'schwiegermutter'

In [18]:
glove_embedding_en.find_analogies('woman', 'love', 'man')

woman - love = man - me


'me'

In [19]:
glove_embedding_de.find_analogies('frau', 'liebe', 'mann')

frau - liebe = mann - mein


'mein'

In [20]:
glove_embedding_en.find_analogies('man', 'motivation', 'woman')

man - motivation = woman - desire


'desire'

In [21]:
glove_embedding_de.find_analogies('mann', 'motivation', 'frau')

mann - motivation = frau - beeinflussen


'beeinflussen'

In [22]:
glove_embedding_en.find_analogies('woman', 'motivation', 'man')

woman - motivation = man - motive


'motive'

In [23]:
glove_embedding_de.find_analogies('frau', 'motivation', 'mann')

frau - motivation = mann - stärke


'stärke'

In [24]:
glove_embedding_en.find_analogies('man', 'respect', 'woman')

man - respect = woman - dignity


'dignity'

In [25]:
glove_embedding_de.find_analogies('mann', 'respekt', 'frau')

mann - respekt = frau - wertschätzung


'wertschätzung'

In [26]:
glove_embedding_en.find_analogies('man', 'success', 'woman')
glove_embedding_de.find_analogies('mann', 'erfolg', 'woman')

man - success = woman - achieved
mann - erfolg = woman - hit


'hit'

In [ ]:
glove_embedding_en.find_analogies('man', 'love', 'woman')
glove_embedding_de.find_analogies('mann', 'liebe', 'frau')

In [ ]:
glove_embedding_en.find_analogies('woman', 'mother in law', 'man')
glove_embedding_de.find_analogies('frau', 'schwiegermutter', 'mann')
glove_embedding_de.find_analogies('mann', 'schwiegervater', 'frau')

In [ ]:
glove_embedding_en.find_analogies('man', 'great', 'woman')
glove_embedding_de.find_analogies('mann', 'großartig', 'frau')

In [ ]:
glove_embedding_en.find_analogies('women', 'kids', 'man')
glove_embedding_de.find_analogies('frau', 'kinder', 'mann')

In [ ]:
glove_embedding_en.find_analogies('man', 'friends', 'woman')
glove_embedding_de.find_analogies('mann', 'freunde', 'frau')

In [ ]:
glove_embedding_en.find_analogies('men', 'friends', 'women')
glove_embedding_de.find_analogies('männer', 'freunde', 'frauen')

In [ ]:
glove_embedding_en.find_analogies('woman', 'safety', 'man')
glove_embedding_de.find_analogies('frau', 'sicherheit', 'mann')

In [ ]:
glove_embedding_en.find_analogies('women', 'orgasm', 'men')
glove_embedding_de.find_analogies('frauen', 'orgasmus', 'männer')

In [ ]:
glove_embedding_en.find_analogies('woman', 'money', 'man')
glove_embedding_de.find_analogies('frau', 'geld', 'mann')

In [ ]:
# klappt einfach
glove_embedding_en.find_analogies('woman', 'man', 'she')
glove_embedding_en.find_analogies('woman', 'man', 'aunt')
glove_embedding_en.find_analogies('woman', 'man', 'sister')
glove_embedding_en.find_analogies('woman', 'man', 'actress')
glove_embedding_en.find_analogies('woman', 'man', 'mother')

In [ ]:
# klappt einfach
glove_embedding_de.find_analogies('frau', 'mann', 'sie')
glove_embedding_de.find_analogies('frau', 'mann', 'tante')
glove_embedding_de.find_analogies('frau', 'mann', 'schwester')
glove_embedding_de.find_analogies('frau', 'mann', 'schauspielerin')
glove_embedding_de.find_analogies('frau', 'mann', 'mutter')

In [ ]:
#klappt einfach
glove_embedding_en.find_analogies('paris', 'france', 'london')
glove_embedding_en.find_analogies('paris', 'france', 'rome')
glove_embedding_en.find_analogies('france', 'paris', 'italy')
glove_embedding_en.find_analogies('florida', 'miami', 'texas')

In [ ]:
glove_embedding_de.find_analogies('frankreich', 'paris', 'london')
glove_embedding_de.find_analogies('frankreich', 'paris', 'rom')
glove_embedding_de.find_analogies('paris', 'frankreich', 'italien')
glove_embedding_de.find_analogies('miami', 'florida', 'texas')

In [ ]:
#klappt einfach
glove_embedding_en.find_analogies('november', 'december', 'march')
glove_embedding_de.find_analogies('november', 'dezember', 'märz')

In [ ]:
#klappt einfach
glove_embedding_en.find_analogies('einstein', 'scientist', 'dali')
glove_embedding_de.find_analogies('einstein', 'wissenschaftler', 'merkel')

In [ ]:
# fun
glove_embedding_en.find_analogies('china', 'rice', 'germany')
glove_embedding_de.find_analogies('china', 'reis', 'deutschland')

In [ ]:
glove_embedding_en.find_analogies('germany', 'beer', 'usa')
glove_embedding_de.find_analogies('deutschland', 'bier', 'usa')

In [ ]:
glove_embedding_en.find_analogies('usa', 'freedom', 'germany')
glove_embedding_de.find_analogies('usa', 'freiheit', 'deutschland')

In [ ]:
glove_embedding_en.find_analogies('builder', 'building', 'politician')
glove_embedding_en.find_analogies('writer', 'book', 'politician')
glove_embedding_de.find_analogies('bauarbeiter', 'baustelle', 'politiker')
glove_embedding_de.find_analogies('schriftsteller', 'buch', 'politiker')

In [ ]:
glove_embedding_en.find_analogies('hacker', 'code', 'politician')
glove_embedding_de.find_analogies('hacker', 'code', 'politiker')

In [ ]:
glove_embedding_en.find_analogies('trump', 'democracy', 'hitler')
glove_embedding_de.find_analogies('trump', 'demokratie', 'hitler')

In [ ]:
glove_embedding_de.get_embedding('trump')

# nearest neighbors

In [ ]:
glove_embedding_en.nearest_neighbors('france')

In [ ]:
glove_embedding_de.nearest_neighbors('frankreich')

In [ ]:
glove_embedding_en.nearest_neighbors('einstein')
glove_embedding_de.nearest_neighbors('einstein')

In [ ]:
glove_embedding_en.nearest_neighbors('trump')
glove_embedding_de.nearest_neighbors('trump')

In [ ]:
glove_embedding_en.nearest_neighbors('merkel')
glove_embedding_de.nearest_neighbors('merkel')

In [ ]:
glove_embedding_en.nearest_neighbors('procrastination')
glove_embedding_de.nearest_neighbors('prokrastination')

In [ ]:
glove_embedding_en.nearest_neighbors('religion')
glove_embedding_de.nearest_neighbors('religion')

In [ ]:
nearest_neighbors('woman',index_to_word, word2vec)
nearest_neighbors('man',index_to_word, word2vec)
nearest_neighbors('nephew',index_to_word, word2vec)

In [ ]:
nearest_neighbors('february',index_to_word, word2vec)
nearest_neighbors('success',index_to_word, word2vec)
nearest_neighbors('money',index_to_word, word2vec)
nearest_neighbors('love',index_to_word, word2vec)

In [ ]:
nearest_neighbors('procrastination', index_to_word, word2vec)